# dataset check 

> Before or after training, we want to check dataset, this checking functionality will be used here

In [ ]:
#| default_exp dataset_check

In [2]:
#| export
import os
import shutil
from IPython.display import display, clear_output
import io
from PIL import Image as Image
from ipywidgets import (widgets, Button,
                        HBox, VBox, Layout)
from ipywidgets import Image as WImage
from pathlib import Path
import numpy as np
from fastcore.all import *
from typing import List, Union, Any, Callable

In [3]:
#| export
from ipywidgets.widgets import Button

In [9]:
#| export
def display_image_row(
        im_path:Union[Path, str],
        move_path:Union[Path, str],
        max_images:int=10,
        start:int=0,
        im_height:int=100,
        im_width:int=100,
    ): 


    Path(move_path).mkdir(exist_ok=True, parents=True)

    def on_next_click(b):
        nonlocal start

        start = start + max_images
        print(f' start = {start}')
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start, 
            im_height,
            im_width
        )
    def on_prev_click(b):
        nonlocal start
        start = max( 0, start - max_images)
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start,
            im_height,
            im_width
        )
    def on_move_click(
            b,
            file_name,
        ):
        source_path = Path(im_path, file_name)
        destination_path = Path(move_path, file_name)
        shutil.move(source_path, destination_path)
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start,
            im_height,
            im_width
        )
    clear_output(wait=True)

    files = sorted(im_path.ls(file_exts=['.png', '.tif']))
    files_to_d = files[start:start + max_images]

    image_boxes = []
    
    for file in files_to_d:
        name_ = Path(file).name
        image_path = Path(im_path, name_)

        with Image.open(image_path) as pil_img:
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='PNG')
            img_widget = WImage(
                value=img_byte_arr.getvalue(),
                format='png',
                width=im_width,
                height=im_height
                )
            
            btn_del = widgets.Button(description='Delete')
            btn_move = widgets.Button(description='Move')

            btn_move.on_click(lambda b,f=name_: on_move_click(b, f))

            box = VBox(
                [
                    img_widget,
                    HBox([btn_del, btn_move])
                    ])
            image_boxes.append(box)


    btn_next = Button(description='Next') 
    btn_prev = Button(description='Prev') 

    if start > 0 : 
        print(f'start = {start}')
        print(f'max = {max_images}')
        btn_prev.on_click(on_prev_click)
    if start + max_images < len(files):
        btn_next.on_click(on_next_click)

    nav = HBox([btn_prev, btn_next])
    display(
        HBox(image_boxes, Layout=Layout(
            flex_flow='row_wrap', 
            align_items='center',
        ))
    )
    display(nav)
    


In [ ]:
data_path = Path(r'N:\homes\hasan\easy_front\overlay_path')
move_path = Path(r'N:\homes\hasan\easy_front\overlay_path_old')


In [5]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

In [6]:
data_path.ls()

(#2) [Path('N:/homes/hasan/easy_front/overlay_path/module_1_side_left_intCycle1_MSA_Cl_0_270912023827_1_2_idx_13.png'),Path('N:/homes/hasan/easy_front/overlay_path/module_1_side_left_intCycle1_MSA_Cl_0_270912023827_1_2_idx_14.png')]

In [10]:
data_path = Path(r'N:\homes\Carsten\Rec23_iwis_Gurt_pins_single_pin')

In [ ]:
display_image_row(
   im_path=data_path, 
   move_path=move_path,
   max_images=10,
   start=0,
   im_height=100,
   im_width=100
)

start = 60
max = 10


In [1]:
#| hide
import nbdev; nbdev.nbdev_export()